In [2]:
import numpy as np, pandas as pd, os
from keras.preprocessing import image
from PIL import Image
import os, random, cv2, copy
from os import path

/Users/anil_adepu/ISIC/_isic-2024-challenge/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [22]:
def return_list(data_path, data_type):
    file_list = [file for file in os.listdir(data_path) if file.lower().endswith(data_type)]
    # print(str(len(file_list)))
    return file_list

def mk_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    return dir_path

data_type = '.jpg'
csv_path = '../train/'
csv_path_permissive = '../FROM_ARCHIVES/train/'
FOLDS = 10
NUM_CLASSES = 2
CLASS_LABELS = [0, 1]


213

In [23]:
train = pd.read_csv(csv_path + 'train_metadata.csv')
cols = train.columns

class_wise = []
for i in CLASS_LABELS:
    class_wise.append(list(train.loc[train.target==i].isic_id))

for x in class_wise:
    random.shuffle(x)

/var/folders/01/fggmj5wx0lvfwymzv_d0m1j40000gn/T/ipykernel_84373/493673462.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(csv_path + 'train_metadata.csv')


In [24]:
train_permissive = pd.read_csv(csv_path_permissive + 'train_metadata_permissive.csv')
cols_permissive = train_permissive.columns

class_wise_permissive = []
for i in CLASS_LABELS:
    class_wise_permissive.append(list(train_permissive.loc[train.target==i].isic_id))

for x in class_wise_permissive:
    random.shuffle(x)

/var/folders/01/fggmj5wx0lvfwymzv_d0m1j40000gn/T/ipykernel_84373/233331253.py:1: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_permissive = pd.read_csv(csv_path_permissive + 'train_metadata_permissive.csv')


In [28]:
def stratify_files(class_wise = []):
    strat=[[] for idx in range(FOLDS)]

    for idx, class_wise_files in enumerate(class_wise):
        cumulative_file_idx = 0
        step = int(np.floor(len(class_wise_files)/FOLDS))
        for fold in range(FOLDS):
            true_file_idx = min(len(class_wise_files), cumulative_file_idx + step)
            strat[fold] += class_wise_files[cumulative_file_idx:true_file_idx]
            cumulative_file_idx = true_file_idx

        # randomly split into folds, here odd/even
        if idx & 2:
            for id, file in enumerate(class_wise_files[cumulative_file_idx:len(class_wise_files)]):
                strat[id].append(file)
        else:
            for id, file in enumerate(class_wise_files[cumulative_file_idx:len(class_wise_files)]):
                strat[len(strat)-1-id].append(file)

        print('----------------------------------------------------------------------------', f"class-{idx}", len(class_wise_files))

    cnt=set()
    for fold in strat:
        for file in fold:
            cnt.add(file)
    print("verification 1: ", len(cnt))

    cnt = 0
    for x in strat:
            cnt += len(x)
    print("verification 2:", cnt)

    for x in strat:
        print(len(x), len(x)*100/len(train))

    return strat

In [29]:
strat = stratify_files(class_wise)
strat_permissive = stratify_files(class_wise_permissive)

---------------------------------------------------------------------------- class-0 400666
---------------------------------------------------------------------------- class-1 393
verification 1:  401059
verification 2: 401059
40105 9.999775594114581
40105 9.999775594114581
40105 9.999775594114581
40105 9.999775594114581
40106 10.00002493398727
40106 10.00002493398727
40106 10.00002493398727
40107 10.000274273859956
40107 10.000274273859956
40107 10.000274273859956
---------------------------------------------------------------------------- class-0 217264
---------------------------------------------------------------------------- class-1 213
verification 1:  217477
verification 2: 217477
21747 5.422394211325516
21747 5.422394211325516
21747 5.422394211325516
21747 5.422394211325516
21747 5.422394211325516
21747 5.422394211325516
21748 5.4226435511982025
21749 5.42289289107089
21749 5.42289289107089
21749 5.42289289107089


In [35]:
train_path='../processed/centre_square_cropped/train/'
train_path_permissive='../processed/permissive/centre_square_cropped/train/'
files = return_list(train_path, '.jpg')
files_permissive = return_list(train_path_permissive, '.jpg')

data_save_path = mk_dir('../processed/stratified/train/stratified_jpg_128/')
data_save_path_permissive = mk_dir('../processed/stratified/train/stratified_jpg_permissive_128/')

In [32]:
for fold, fold_files in enumerate(strat):
    fold_wise_data_save_path = mk_dir(data_save_path + 'train' + str(fold) + '/')
    idx = 0
    for file in fold_files:
        idx += 1
        file_name = file + '.jpg'
        if idx % 1000 == 0:
            print('Processing Img {idx}: {file_name}'.format(idx=idx + 1, file_name=file_name))
        org_img = np.asarray(image.load_img(train_path + file_name))
        org_img = Image.fromarray(org_img)
        org_img.save(path.join(fold_wise_data_save_path + file_name))

In [36]:
for fold, fold_files in enumerate(strat_permissive):
    fold_wise_data_save_path_permissive = mk_dir(data_save_path_permissive + 'train' + str(fold) + '/')
    idx = 0
    for file in fold_files:
        idx += 1
        file_name = file + '.jpg'
        if idx % 1000 == 0:
                print('Processing Img (permissive) {idx}: {file_name}'.format(idx=idx + 1, file_name=file_name))
        org_img = np.asarray(image.load_img(train_path_permissive + file_name))
        org_img = Image.fromarray(org_img)
        org_img.save(path.join(fold_wise_data_save_path_permissive + file_name))

Processing Img (permissive) 1001: ISIC_6466578.jpg
Processing Img (permissive) 2001: ISIC_0518184.jpg
Processing Img (permissive) 3001: ISIC_6954845.jpg
Processing Img (permissive) 4001: ISIC_6878138.jpg
Processing Img (permissive) 5001: ISIC_3424564.jpg
Processing Img (permissive) 6001: ISIC_3936032.jpg
Processing Img (permissive) 7001: ISIC_3014182.jpg
Processing Img (permissive) 8001: ISIC_8303426.jpg
Processing Img (permissive) 9001: ISIC_9819921.jpg
Processing Img (permissive) 10001: ISIC_9881031.jpg
Processing Img (permissive) 11001: ISIC_0391968.jpg
Processing Img (permissive) 12001: ISIC_3240652.jpg
Processing Img (permissive) 13001: ISIC_4686430.jpg
Processing Img (permissive) 14001: ISIC_9356126.jpg
Processing Img (permissive) 15001: ISIC_6607270.jpg
Processing Img (permissive) 16001: ISIC_1980527.jpg
Processing Img (permissive) 17001: ISIC_9502083.jpg
Processing Img (permissive) 18001: ISIC_4532474.jpg
Processing Img (permissive) 19001: ISIC_1821845.jpg
Processing Img (permi